# Analysing the redundancies of User Stories benefits

In [1]:
import locale
import os
from itertools import combinations

import pandas as pd
import tiktoken
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter as utils_get_column_letter
#from openpyxl.worksheet.worksheet import Worksheet

from support_functions.excel_helper import save_to_excel
from support_functions.load_data import load_datasets_with_annotations as loading
from support_functions.request_handler import process_user_stories, process_user_stories_parallel
from support_functions.data_transformations import convert_annotation_dataset
from support_functions.time_recorder import TimeRecorder

In [2]:
locale.setlocale(locale.LC_ALL, 'de_DE') 
MODEL_VERSION_NAME = os.getenv('MODEL_VERSION')
THREADING = bool(int(os.getenv('THREADING')))
print(f"Threading is used: {THREADING}")
print(MODEL_VERSION_NAME)

Threading is used: False
gpt-3.5-turbo


In [3]:
time_recorders: dict = {}
time_recorder: TimeRecorder = None

### Load all User Stories from the different packages

We do not need:
- POS

Highlighting words with #

In [4]:
datasets: dict[str, list] = loading()
print(datasets["g02"][0])

{'PID': '#G02#', 'Text': '#G02# As a Data user, I want to have the 12-19-2017 deletions processed.', 'Persona': ['Data user'], 'Action': {'Primary Action': ['have'], 'Secondary Action': ['processed']}, 'Entity': {'Primary Entity': ['12-19-2017 deletions'], 'Secondary Entity': ['']}, 'Benefit': '', 'Triggers': [['Data user', 'have']], 'Targets': [['processed', '12-19-2017 deletions'], ['have', '12-19-2017 deletions']], 'Contains': [], 'Persona POS': {'Persona POS tag': [['NOUN', 'NOUN']], 'Persona POS text': [['Data', 'user']]}, 'Action POS': {'Primary Action POS': {'Primary Action POS tag': [['VERB']], 'Primary Action POS text': [['have']]}, 'Secondary Action POS': {'Secondary Action POS tag': [['VERB']], 'Secondary Action POS text': [['processed']]}}, 'Entity POS': {'Primary Entity POS': {'Primary Entity POS tag': [['NUM', 'NOUN']], 'Primary Entity POS text': [['12-19-2017', 'deletions']]}, 'Secondary Entity POS': {'Secondary Entity POS tag': [[]], 'Secondary Entity POS text': [[]]}}}

In [5]:
datasets = convert_annotation_dataset(datasets)

### Prepaire Prompting

In [6]:
definition_json_format_pair_user_stories: str = '''
{
    "relatedStories": [1, 3],
    "redundantMainPart": true, 
    "mainPartRedundancies": [
        fullRedundancy: true,
        fullRedundancy: [
            {
                "reasonDescription": "The redundancy arises from the repetition of the concept of user verification expressed in slightly different ways for emphasis or to cater to different audiences.",
                "referenceToOriginalText": ["As a #User#, I #want# to #verify# myself", "As an application #User#, I #want# to #verify# that I am who I am"]
                "pairsOfRedundancy": {
                    ["User", "application #User#"],
                    ["verify", "verify"]
                }
            },
        ],
        partialRedundancy: true,
        partialRedundancy: [
            {
                "reasonDescription": "The redundancy arises from the repetition of the concept of user verification expressed in slightly different ways for emphasis or to cater to different audiences.",
                "referenceToOriginalText": ["As a User, I want to verify myself", "As an application User, I want to verify that I am who I am"]
            },
        ]        
    ],
    "redundantBenefit": true, 
    "benefitRedundancies": [
        fullRedundancy: true,
        fullRedundancy: [
            {
                "reasonDescription": "The redundancy arises from the repetition of the concept of user verification expressed in slightly different ways for emphasis or to cater to different audiences.",
                "referenceToOriginalText": ["As a User, I want to verify myself", "As an application User, I want to verify that I am who I am"]
            },
        ],
        partialRedundancy: true,
        partialRedundancy: [
            {
                "reasonDescription": "The redundancy arises from the repetition of the concept of user verification expressed in slightly different ways for emphasis or to cater to different audiences.",
                "referenceToOriginalText": ["As a User, I want to verify myself", "As an application User, I want to verify that I am who I am"]
            },
        ] 
    ]
}
'''